# IMDB Movie Data

### Jordan Miranda

In this notebook we'll be looking at a small dataset from IMDB that contains different movies with information about the movie such as genre, duration, country, etc.

The goal within this notebook is to clean the dataset and prepare it for possible feature engineering and modelling later on. To clean this data we'll need to address any null values, clean/standardize values across the dataset, and possibly renaming columns to appropriately reflect the data they contain.

We'll get started by importing the standard tools used for data manipulation with Python - `pandas` and `NumPy`. 
If any data visualization is necessary we'll import `Matplotlib` and `seaborn`.

In [1]:
# importing standard data tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


Next we'll need to import the data we'll be working with.

In [2]:
# Loading in our dataset, including the delimiter used to separate the values
movies_df = pd.read_csv("data/messy_IMDB_dataset.csv", sep=";")
movies_df.head()

,IMBD title ID,Original titl�,Release year,Genr�,Duration,Country,Content Rating,Director,Unnamed: 8,Income,Votes,Score
0,tt0111161,The Shawshank Redemption,1995-02-10,Drama,142,USA,R,Frank Darabont,NaN,$ 28815245,2.278.845,9.3
1,tt0068646,The Godfather,09 21 1972,"Crime, Drama",175,USA,R,Francis Ford Coppola,NaN,$ 246120974,1.572.674,9.2
2,tt0468569,The Dark Knight,23 -07-2008,"Action, Crime, Drama",152,US,PG-13,Christopher Nolan,NaN,$ 1005455211,2.241.615,9.
3,tt0071562,The Godfather: Part II,1975-09-25,"Crime, Drama",220,USA,R,Francis Ford Coppola,NaN,"$ 4o8,035,783",1.098.714,"9,.0"
4,tt0110912,Pulp Fiction,1994-10-28,"Crime, Drama",,USA,R,Quentin Tarantino,NaN,$ 222831817,1.780.147,"8,9f"


## Introductory Information on the Dataset

Now that we've loaded the dataset, let's gather some basic info about the data such as its shape, datatypes, missing values (if any), and so on.

In [3]:
movies_df.shape

(101, 12)

This dataset contains 101 rows with 12 columns. Let's look at the names of the columns and their datatypes.

In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IMBD title ID   100 non-null    object 
 1   Original titl�  100 non-null    object 
 2   Release year    100 non-null    object 
 3   Genr�           100 non-null    object 
 4   Duration        99 non-null     object 
 5   Country         100 non-null    object 
 6   Content Rating  77 non-null     object 
 7   Director        100 non-null    object 
 8   Unnamed: 8      0 non-null      float64
 9   Income          100 non-null    object 
 10   Votes          100 non-null    object 
 11  Score           100 non-null    object 
dtypes: float64(1), object(11)
memory usage: 9.6+ KB


## Initial Problems

Here are the 11 columns (index excluded) that are in this dataset. Using `.info()` we can spot a few problems with the data already. Below we'll identify some of the initial concerns, with no particular priority (currently).

### Datatypes

There are 10 columns with a datatype of `object` and 1 column having a datatype of `float`. From looking at the `.head()` we can see that the datatypes for some of these columns shouldn't be `object` but due to poor data entry formatting they've defaulted to `object`. 

One of the major tasks with this dataset will be to reformat the data entries so that they are consistent across each column and once done, assign the correct datatype to each respective column.

### Missing Values

Looking across some columns we can see there are a few missing values. The column with the most missing values is `Unnamed: 8` with every value missing, second is `Content Rating` with 34 missing values. Some external research may need to be performed to source Content Ratings or we can impute the values based on EDA.

### Inconsistent Data Entry Formatting

Looking at `Release year` we can see that the entries in this column vary in their ordering of MM/DD/YYYY. Looking at the `.head()` in the beginning of the dataset we also see inconsistent formatting with `Country`, `Income`, `Votes`, and `Score`. 

### Inaccurate/Misspelled Column Titles

Columns `Original title` and `Genre` contain characters that aren't displayed properly so we'll need to rename them to be readable.  We'll also need to correct the spelling of IMDB in the first column `IMBD title ID`.

We can also see the `Release year` column isn't accurately representing the data in the column. The data within the column contains not just the release year but also the day and month, therefore the column's appropriate title would be `Release Date`. This will be the first problem we'll address as it's a simple fix.

## Cleaning

Now that we've identified some of the initial problems, we can start cleaning the data set. If during our cleaning we discover new issues we will make note of the issue and address it later.

### Columns

As mentioned earlier, fixing the column titles will be one of the simpler fixes so we'll take care of that now.


In [5]:
# Getting all of the columns
movies_df.columns

Index(['IMBD title ID', 'Original titl�', 'Release year', 'Genr�', 'Duration',
       'Country', 'Content Rating', 'Director', 'Unnamed: 8', 'Income',
       ' Votes ', 'Score'],
      dtype='object')

In [6]:
# Renaming the columns mentioned earlier
# using .rename and providing a dictionary of the columns that need fixing
movies_df.rename(columns={
    "IMBD title ID": "IMDB Title ID",
    movies_df.columns[1]: "Movie Title",
    "Release year": "Release Date",
    movies_df.columns[3]: "Genre",
    "Income": "Revenue",
    " Votes ": "Number of Votes"}, # Renaming income & votes as well for added clarity
inplace=True)

In [7]:
# Checking to see the renaming was done properly
movies_df.sample(3)
# Looks good!

,IMDB Title ID,Movie Title,Release Date,Genre,Duration,Country,Content Rating,Director,Unnamed: 8,Revenue,Number of Votes,Score
91,tt0086879,Amadeus,1985-02-15,"Biography, Drama, History",160,USA,R,Milos Forman,NaN,$ 52066791,361.028,7.6
99,tt1832382,Jodaeiye Nader az Simin,2011-10-21,Drama,123,Iran,PG-13,Asghar Farhadi,NaN,$ 22926076,214.165,7.4
37,tt0103064,Terminator 2: Judgment Day,1991-12-19,"Action, Sci-Fi",137,USA,R,James Cameron,NaN,$ 520884847,974.970,8.4


We've successfully renamed our problem columns. However, we do see there's 1 column leftover named `Unnamed: 8`. The entire column consists of null values and it's hard to identify without external context what this column would've been in this dataset. We're likely going to drop this column as there's no way we'll be able to impute any values into this column.

Before dropping we'll confirm the column is full of null values.

In [8]:
# Finding number of null values in the unnamed column
unnamed_nulls = movies_df["Unnamed: 8"].isnull().sum()

# Taking the number of rows in the dataset
num_of_rows = movies_df.shape[0]

# If this returns True, all values in the unnamed column are null
unnamed_nulls == num_of_rows

True

We've double checked the column consists of all Null values. Let's drop the column.

In [9]:
# Dropping the column
movies_df.drop(columns="Unnamed: 8", inplace=True)

# Checking to see the drop was done successfully
movies_df.sample(3)
# Looks good!

,IMDB Title ID,Movie Title,Release Date,Genre,Duration,Country,Content Rating,Director,Revenue,Number of Votes,Score
100,tt0045152,Singin' in the Rain,1953-02-05,"Comedy, Musical, Romance",103,USA,NaN,Stanley Donen,$ 1864182,213.152,7.4
68,tt0087843,Once Upon a Time in America,1984-09-28,"Crime, Drama",229,USA,R,Sergio Leone,$ 5472914,302.317,8.0
30,tt0047478,Shichinin no samurai,1955-08-19,"Action, Adventure, Drama",207,Japan,Unrated,Akira Kurosawa,$ 322773,307.958,8.6


We've successfully dropped the unnamed column. Let's continue on to missing values.

### Missing Values

Now that we've dealt with the columns, let's approach our next problem - missing values.

In [10]:
# Getting the number of null values in each column
movies_df.isna().sum()

IMDB Title ID       1
Movie Title         1
Release Date        1
Genre               1
Duration            2
Country             1
Content Rating     24
Director            1
Revenue             1
Number of Votes     1
Score               1
dtype: int64

In [11]:
# Locating the rows that contain null values
null_rows = movies_df.isnull().any(axis=1)

# Displaying the rows
movies_df[null_rows]

,IMDB Title ID,Movie Title,Release Date,Genre,Duration,Country,Content Rating,Director,Revenue,Number of Votes,Score
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,tt0133093,The Matrix,1999-05-07,"Action, Sci-Fi",NaN,USA,R,"Lana Wachowski, Lilly Wachowski",$ 465718588,1.632.315,++8.7
27,tt0118799,La vita B9 bella,1997-12-20,"Comedy, Drama, Romance",116,Italy1,NaN,Roberto Benigni,$ 230098753,605.648,8.6
28,tt6751668,Gisaengchung,2019-11-07,"Comedy, Drama, Thriller",132,South Korea,NaN,Bong Joon Ho,$ 257604912,470.931,8.6
36,tt0110413,Léon,1995-04-07,"Action, Crime, Drama",110,France,NaN,Luc Besson,$ 19552639,1.007.598,8.5
40,tt7286456,Joker,2019-10-03,"Crime, Drama, Thriller",122,USA,NaN,Todd Phillips,$ 1074251311,855.097,8.4
41,tt1675434,Intouchables,2012-02-24,"Biography, Comedy, Drama",112,France,NaN,"Olivier Nakache, Éric Toledano",$ 426588510,736.691,8.4
47,tt0095327,Hotaru no haka,2015-10-11,"Animation, Drama, War",89,Japan,NaN,Isao Takahata,$ 516962,225.438,8.3
48,tt0095765,Nuovo Cinema Paradiso,1988-11-17,Drama,155,Italy,NaN,Giuseppe Tornatore,$ 13826605,223.050,8.3
56,tt4154756,Avengers: Infinity War,2018-04-25,"Action, Adventure, Sci-Fi",149,USA,NaN,"Anthony Russo, Joe Russo",$ 2048359754,796.486,8.2


There's one row that consists of entirely `NaN` values, we'll drop the column before dealing with the other `NaN` values in the `Content Rating` column.

In [12]:
# Dropping the row consisting of all NaNs
movies_df.drop(index=13, inplace=True)

# Resetting the index
movies_df.reset_index(drop=True, inplace=True)

# Seeing the remaining null values across all columns
movies_df.isna().sum()

IMDB Title ID       0
Movie Title         0
Release Date        0
Genre               0
Duration            1
Country             0
Content Rating     23
Director            0
Revenue             0
Number of Votes     0
Score               0
dtype: int64

After dropping our row of all `NaN` values we're left with 1 null in the `Duration` column and 23 nulls in the `Content Rating` column.

Since it's only one missing `Duration` we'll go ahead and pull the duration of the movie from Google and replace the null value directly.

In [13]:
# Finding the movie with a missing duration
movies_df[movies_df['Duration'].isna()]

,IMDB Title ID,Movie Title,Release Date,Genre,Duration,Country,Content Rating,Director,Revenue,Number of Votes,Score
13,tt0133093,The Matrix,1999-05-07,"Action, Sci-Fi",NaN,USA,R,"Lana Wachowski, Lilly Wachowski",$ 465718588,1.632.315,++8.7


Both Google and IMDB show that the movies duration was 2 hours and 16 minutes. The durations of the movies in the `Duration` column are formatted in minutes so we'll do the same.

In [14]:
# Calculating the matrix duration in minutes
matrix_duration = (60 * 2) + 16

# Inputting the matrix duration into the entry within the dataframe
movies_df["Duration"][movies_df.index == 13] = matrix_duration

# Checking that it worked
movies_df[movies_df.index == 13]

,IMDB Title ID,Movie Title,Release Date,Genre,Duration,Country,Content Rating,Director,Revenue,Number of Votes,Score
13,tt0133093,The Matrix,1999-05-07,"Action, Sci-Fi",136,USA,R,"Lana Wachowski, Lilly Wachowski",$ 465718588,1.632.315,++8.7


We've successfully filled in `The Matrix`'s duration. Let's check what remains for missing values now.

In [22]:
movies_df.isna().sum()

IMDB Title ID       0
Movie Title         0
Release Date        0
Genre               0
Duration            0
Country             0
Content Rating     23
Director            0
Revenue             0
Number of Votes     0
Score               0
dtype: int64

The only column left containing null values is `Content Rating` with 23 null values.

Since there are 23 null values, manually inputting each movie's content rating would be quite a tedious process as it makes up just under a quarter of all the data. If this dataset were much larger, sourcing 25% of the missing values and inputting them into a dataset wouldn't be feasible. For best practices, will try other methods before manual imputation.

The first proposal to fill in these values will be to scrape the movie's content rating from IMDB. When going on IMDB's website, we can see that any given movie URL is in the following format: `imdb.com/title/(IMDB Title ID Here)/`. With each entry in our dataset having an `IMDB Title ID`, we can attach the `IMDB Title ID` to the imdb url to get the webpage and from there scrape for the movie's `Content Rating`.

Let's begin.

To start, we'll need to import BeautifulSoup, our webscraper of choice.

In [16]:
# Importing scraping libraries
import requests
from bs4 import BeautifulSoup

# Importing time to ensure we don't get rate limited
import time

In [28]:
# Ping IMDB page for the matrix
response = requests.get("https://www.imdb.com/title/tt0133093/")

# Turn the undecoded content into BS object 
soup = BeautifulSoup(response.content)

# Let's look at it
#soup
# I've commented out the soup here because the text in the response is incredibly long.
# If you'd like to see what it looks like yourself you can remove the # on the soup line and re-run this cell.

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/"><head><script></script><script>if(typeof uet === 'function'){ uet('bb', 'LoadAds', {wb: 1}); }</script><script>if(typeof uet === 'function'){ uet('be', 'LoadAds', {wb: 1}); }</script><script>if(typeof uex === 'function'){ uex('ld', 'LoadAds', {wb: 1}); }</script><script>if(typeof uet === 'function'){ uet('bb', 'LoadStyledComponents', {wb: 1}); }</script><style data-styled="" data-styled-version="5.2.1">.haXymx{-webkit-flex-shrink:0;-ms-flex-negative:0;flex-shrink:0;display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-pack:center;-webkit-justify-content:center;-ms-flex-pack:center;justify-content:center;-webkit-user-select:none;-moz-user-select:none;-ms-user-select:none;user-select:none;margin-left:0.25rem;margin-right:auto;-webkit-order

In [18]:
# Looking at the HTML, the movie's duration and rating are in a list class
# We'll pull all lists with this class and take the 2nd and 3rd items in the list through slicing
info_list = soup.find_all("li", {"class": "ipc-inline-list__item"})

# Slicing the 2 li elements of importance and assigning them to a new variable
year_rating_duration = info_list[1:3]

# Looking into the contents of the 2 li elements
year_rating_duration


[<li class="ipc-inline-list__item" role="presentation"><a class="ipc-link ipc-link--baseAlt ipc-link--inherit-color TitleBlockMetaData__StyledTextLink-sc-12ein40-1 rgaOW" href="/title/tt0133093/parentalguide/certificates?ref_=tt_ov_pg">14A</a><span class="TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex">14A</span></li>,
 <li class="ipc-inline-list__item" role="presentation">2<!-- -->h<!-- --> <!-- -->16<!-- -->m</li>]

In [19]:
variable_rating = str(year_rating_duration[0]).split(">")[2]

matrix_rating = variable_rating.split("<")[0]

matrix_rating

'14A'

In [20]:
r_d_header_test2 = soup.find_all("span", {"class": "TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex"})
r_d_header_test2

[<span class="TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex">1999</span>,
 <span class="TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex">14A</span>]